In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the data

In [3]:
list_df = pd.read_csv('girlgroup_list.csv')
list_df.head()

,Year,Artists Name,English Name,Category,Artist ID
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,2007.0,소녀시대,Girls' Generation,Idol,80026432.0
3,2007.0,원더걸스,Wonder Girls,Idol,80022182.0
4,2007.0,카라,KARA,Idol,80023353.0


In [5]:
list_df.dropna(inplace=True)
list_df

,Year,Artists Name,English Name,Category,Artist ID
2,2007.0,소녀시대,Girls' Generation,Idol,80026432.0
3,2007.0,원더걸스,Wonder Girls,Idol,80022182.0
4,2007.0,카라,KARA,Idol,80023353.0
6,2009.0,투에니원,2NE1,Idol,80054982.0
9,2009.0,에프엑스,f(x),Idol,80059218.0
14,2010.0,걸스데이,Girl's Day,Idol,80073726.0
15,2010.0,미쓰에이,miss A,Idol,80073013.0
18,2011.0,에이핑크,Apink,Idol,80097978.0
22,2012.0,EXID,EXID,Idol,80120879.0
24,2012.0,AOA,AOA,Idol,80132659.0


In [7]:
group_ids = list_df[list_df['Category']=='Idol']['Artist ID'].tolist() 
solo_ids = list_df[list_df['Category']=='Solo']['Artist ID'].tolist()

In [12]:
group_ids = [int(id_) for id_ in group_ids]
solo_ids = [int(id_) for id_ in solo_ids]

## Lyrics csv


In [13]:
def lyrics_scraping(artist_id_):
    # 1. url 받아오기
    # def scrape_bugs(artist_id, page_numbers):
    # for page_number in range(1,page_numbers+1):

    url = f'https://music.bugs.co.kr/artist/{artist_id_}/tracks?type=RELEASE'
    request = requests.get(url) # url 불러오기 요청
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')

    song_list = soup.find("table", attrs={'class':'list trackList'}) # 해당 연도의 곡 리스트
    # 해당 url에서 class가 list trackList인 'table' 태그의 데이터를 song_list에 담음

    titles = soup.select('p.title') # p 태그의 클래스가 title인 데이터
    artists = soup.select('p.artist') # p 태그의 클래스가 artist인 데이터
    albums = soup.select('td.left > a.album') # td 태그의 클래스가 left이고, 하위 데이터 중 a 태그의 클래스가 album인 데이터

    song_title = [] # 노래 제목
    song_artist = [] # 가수명
    album_list = [] # 앨범명
    like_counts = [] # 좋아요 수
    song_lyrics = [] # 가사
    album_img_url = [] # 앨범 이미지 url
    id_url_list = [] # 노래 고유 번호가 있는 href 데이터를 담는 리스트
    id_list = [] # 노래 고유 번호를 담는 리스트


    for elem in soup.find_all('a', href=re.compile('https://music.bugs.co.kr/track')): # 해당 url에서 a 태그를 전부 찾기
        id_url_list.append(elem['href']) # a 태그 중에서 href 데이터만 따로 빼내서 id_url_list에 담기

    for j in range(len(id_url_list)):
        a = id_url_list[j].lstrip('https://music.bugs.co.kr/track/')  # 링크1 제거
        # lstrip() : 데이터의 왼쪽에서 괄호 안 데이터의 패턴이 발견되면 제거

        a1 = a.rstrip('wl_ref=list_tr_08_tr')  # 링크2 제거
        # rstrip() : 데이터의 오른쪽에서 괄호 안 데이터의 패턴이 발견되면 제거
        b = a1.rstrip('?')  # 물음표 제거

        id_list.append(b) 

    # 타이틀, 가수, 앨범명
    for k in range(len(id_list)):
        rank = k + 1 # k=0부터 시작

        title = titles[k].text.strip().split('\n')[0] # titles 데이터에서 양쪽 공백을 제거하고, \n을 기준으로 분리
        song_title.append(title) #song_title에 추가

        artist = artists[k].text.strip().split('\n')[0] # artists 데이터에서 양쪽 공백을 제거하고, \n을 기준으로 분리
        song_artist.append(artist) #song_artist에 추가

        album = albums[k].text.strip().split('\n')[0] # albums 데이터에서 양쪽 공백을 제거하고, \n을 기준으로 분리
        album_list.append(album) #album_list에 추가



    # 좋아요 & 가사 & 앨범 이미지(url)
    for m in range(len(id_list)):
        l_url = 'https://music.bugs.co.kr/track/' + str(id_list[m]) + '?wl_ref=list_tr_08_' # 전체 url 생성 : 곡 상세 페이지
        l_request = requests.get(l_url)
        l_html = l_request.text
        l_soup = BeautifulSoup(l_html, "html.parser")

        # 좋아요
        like_count = str(l_soup.select(
            '#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span > em')) # 좋아요 데이터가 있는 html 구조
        like_count = re.sub('<.+?>', '', like_count, 0) # 특수문자 제거
        like_counts.append(like_count) # 리스트에 추가

        # 가사
        lyric = str(l_soup.select(
            '#container > section.sectionPadding.contents.lyrics > div.innerContainer > div.lyricsContainer > p > xmp'))
        # 가사 데이터가 있는 html 구조 #container > section.sectionPadding.contents.lyrics > div > div > xmp
        lyric = re.sub('<.+?>', '', lyric, 0)
        song_lyrics.append(lyric)

        #     # 앨범 이미지
        #     a_img = l_soup.find("li", class_="big").find("img") # li 태그의 class가 big인 곳에서, img 태그를 가진 것 찾기
        #     img_src = a_img.get('src') # img 태그에서 src 데이터를 img_src에 담음
        #     album_img_url.append(img_src) # 리스트에 추가

    column_list = { # dict-key
        'song_id': id_list,
        'song_name': song_title,
        'artist': song_artist,
        'album': album_list,
        'Like_Count': like_counts,
        'Lyric': song_lyrics,
    }

    df = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df = df.transpose() # 행과 열 전환

    return df.to_csv(f'data/{artist_id_}.csv') # 편집자 실행


## Album csv

In [16]:
def album_scraping(artist_id_):
    url = f'https://music.bugs.co.kr/artist/{artist_id_}/albums'
    request = requests.get(url)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')

    artist_name = []
    album_id = []
    album_title = []
    album_release_date=[]

    for elem in soup.find_all('div', attrs={'class': 'albumTitle'}):
        album_id.append(elem.select('a')[0]['href'].lstrip('https://music.bugs.co.kr/album/').split('?')[0])
        album_title.append(elem.select('a')[0]['title'])

    for elem in soup.find_all('div', attrs={'class': 'subInfo'}):
        try:
            artist_name.append(elem.select('a')[0].get_text())
            album_release_date.append(elem.select('time')[0].get_text())
        except:
            artist_name.append(elem.select('span')[0].get_text())
            album_release_date.append(elem.select('time')[0].get_text())

    artist_id = [artist_id_ for _ in range(len(album_id))]

    column_list = {
        'artist_id': artist_id,
        'artist_name': artist_name,
        'album_id': album_id,
        'album_title': album_title,      
        'release_date': album_release_date
        }

    df1 = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df1 = df1.transpose() # 행과 열 전환 
    df1['artist_id'] = artist_id_

    return df1.to_csv(f'data/albums/{artist_id_}_album.csv') # 편집자 실행

In [17]:
%%time
for ids in group_ids:
    album_scraping(ids)

CPU times: user 2.69 s, sys: 80.9 ms, total: 2.77 s
Wall time: 1min 8s


In [18]:
%%time
for ids in solo_ids:
    album_scraping(ids)

CPU times: user 1.18 s, sys: 27.2 ms, total: 1.21 s
Wall time: 30.8 s


-----

## album_and_songs df

In [23]:
def album_and_songs(artist_id_):

    album_df = pd.read_csv(f'data/albums/{artist_id_}_album.csv')
    album_id_lst = album_df.album_id.to_list()

    album_id = []
    song_id = []
    song_title = []

    for album_id_ in album_id_lst:
        url_album = f'https://music.bugs.co.kr/album/{album_id_}'
        request_album = requests.get(url_album)
        html_album = request_album.text
        soup_album = BeautifulSoup(html_album, 'html.parser')

        album_song_list=soup_album.find('table', attrs = {'class': 'list trackList byAlbum'})

        for elem in album_song_list.find_all('p', attrs={'class':'title'}):
            try:            
                song_id.append(elem.select('a')[0]['onclick'].split('.listen(')[1].split(',')[0][1:-1])
                song_title.append(elem.get_text().strip('\n'))
                album_id.append(album_id_)

            except:
                album_id.append(album_id_)
                song_id.append('-')
                song_title.append(elem.get_text().replace('\n', ' ').strip())

    column_list = {
            'album_id': album_id,
            'song_id': song_id,      
            'song_title': song_title
            }

    df1 = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df1 = df1.transpose() # 행과 열 전환 

    return df1.to_csv(f'data/albums/{artist_id_}_album_and_songs.csv') # 편집자 실행


In [25]:
%%time
for ids in group_ids[:13]:
    album_and_songs(ids)

CPU times: user 47.3 s, sys: 831 ms, total: 48.1 s
Wall time: 12min 42s


In [28]:
%%time
for ids in group_ids[13:]:
    album_and_songs(ids)

CPU times: user 26.6 s, sys: 530 ms, total: 27.2 s
Wall time: 8min 51s


In [37]:
%%time
for ids in solo_ids:
    album_and_songs(ids)

CPU times: user 36.7 s, sys: 712 ms, total: 37.4 s
Wall time: 11min 13s


## Merge - 1 (Album & Songs)

In [38]:
def merge_album_songs_df(artist_id_):
    al_ = pd.read_csv(f'data/albums/{artist_id_}_album.csv')
    al_and_song = pd.read_csv(f'data/albums/{artist_id_}_album_and_songs.csv')
    merged_df = al_and_song.merge(al_, how='left', on='album_id')[['artist_id', 'artist_name', 'album_id', 'album_title', 'song_id', 'song_title', 'release_date']]
    return merged_df.to_csv(f'data/tracks/{artist_id_}_merged.csv')

In [39]:
%%time
for ids in group_ids:
    merge_album_songs_df(ids)

CPU times: user 193 ms, sys: 18.7 ms, total: 211 ms
Wall time: 236 ms


In [40]:
%%time
for ids in solo_ids:
    merge_album_songs_df(ids)

CPU times: user 250 ms, sys: 8.19 ms, total: 259 ms
Wall time: 263 ms


## Lyrics

In [43]:
def lyrics_scraping(artist_id_):
    
    song_df = pd.read_csv(f'data/tracks/{artist_id_}_merged.csv')
    song_df = song_df[song_df['song_id']!='-']
    song_id_lst1 = list(set(song_df.song_id.to_list()))

    song_id_lst = []
    lyric_lst = []
    artist_name_lst = []


    for i in song_id_lst1:
        l_url = f'https://music.bugs.co.kr/track/{i}'
        l_request = requests.get(l_url)
        l_html = l_request.text
        l_soup = BeautifulSoup(l_html, 'html.parser')
        
        try:
            artist_name = l_soup.select('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(1) > td > a')[0].getText().strip('\r\n\t')
        except:
            artist_name = '-'

        lyric = str(l_soup.select(
                '#container > section.sectionPadding.contents.lyrics > div.innerContainer > div.lyricsContainer > p > xmp'))
            # 가사 데이터가 있는 html 구조 #container > section.sectionPadding.contents.lyrics > div > div > xmp
        lyric = re.sub('<.+?>', '', lyric, 0)

        song_id_lst.append(i)
        artist_name_lst.append(artist_name)
        lyric_lst.append(lyric)


    column_list = { # dict-key
            'song_id': song_id_lst,
            'song_artist': artist_name_lst,
            'song_lyrics': lyric_lst,
        }

    df = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df = df.transpose() # 행과 열 전환


    return df.to_csv(f'data/lyrics/{artist_id_}_lyrics.csv') # 편집자 실행

In [44]:
%%time
for ids in group_ids:
    lyrics_scraping(ids)

CPU times: user 4min 34s, sys: 5.93 s, total: 4min 40s
Wall time: 1h 41min 56s


In [45]:
%%time
for ids in solo_ids:
    lyrics_scraping(ids)

CPU times: user 2min 44s, sys: 3.57 s, total: 2min 48s
Wall time: 1h 29s


## 앨범 2페이지까지 있는 애들 2차

In [30]:
# #Boa
# artist_id_ = '1168'
# url = 'https://music.bugs.co.kr/artist/1168/albums?type=RELEASE&sort=R&row=70&page=2'
# request = requests.get(url)
# html = request.text
# soup = BeautifulSoup(html, 'html.parser')

# artist_name = []
# album_id = []
# album_title = []
# album_release_date=[]

# for elem in soup.find_all('div', attrs={'class': 'albumTitle'}):
#     album_id.append(elem.select('a')[0]['href'].lstrip('https://music.bugs.co.kr/album/').split('?')[0])
#     album_title.append(elem.select('a')[0]['title'])

# for elem in soup.find_all('div', attrs={'class': 'subInfo'}):
#     try:
#         artist_name.append(elem.select('a')[0].get_text())
#         album_release_date.append(elem.select('time')[0].get_text())
#     except:
#         artist_name.append(elem.select('span')[0].get_text())
#         album_release_date.append(elem.select('time')[0].get_text())

# artist_id = [artist_id_ for _ in range(len(album_id))]

# column_list = {
#     'artist_id': artist_id,
#     'artist_name': artist_name,
#     'album_id': album_id,
#     'album_title': album_title,      
#     'release_date': album_release_date
#     }

# df1 = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
# df1 = df1.transpose() # 행과 열 전환 
# df1['artist_id'] = artist_id_
# df1.to_csv(f'data/albums/1168_album_2.csv') # 편집자 실행

In [36]:
# boa1 = pd.read_csv('data/albums/1168_album_2.csv')
# boa2=pd.read_csv('data/albums/1168_album.csv')
# boa_final = pd.concat([boa1,boa2]).drop(columns='Unnamed: 0').reset_index()
# boa_final.to_csv(f'data/albums/1168_album.csv')

In [369]:
def album_scraping2(artist_id_):
    url = f'https://music.bugs.co.kr/artist/{artist_id_}/albums?type=RELEASED&page=2'
    request = requests.get(url)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')

    artist_name = []
    album_id = []
    album_title = []
    album_release_date=[]

    for elem in soup.find_all('div', attrs={'class': 'albumTitle'}):
        album_id.append(elem.select('a')[0]['href'].lstrip('https://music.bugs.co.kr/album/').split('?')[0])
        album_title.append(elem.select('a')[0]['title'])

    for elem in soup.find_all('div', attrs={'class': 'subInfo'}):
        try:
            artist_name.append(elem.select('a')[0].get_text())
            album_release_date.append(elem.select('time')[0].get_text())
        except:
            artist_name.append(elem.select('span')[0].get_text())
            album_release_date.append(elem.select('time')[0].get_text())

    artist_id = [artist_id_ for _ in range(len(album_id))]

    column_list = {
        'artist_id': artist_id,
        'artist_name': artist_name,
        'album_id': album_id,
        'album_title': album_title,      
        'release_date': album_release_date
        }

    df1 = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df1 = df1.transpose() # 행과 열 전환 
    df1['artist_id'] = artist_id_

    return df1.to_csv(f'data-sm_vs_yg/albums/{artist_id_}_album_2_pages.csv') # 편집자 실행

In [371]:
album_scraping2(64843)

In [372]:
def album_and_songs2(artist_id_):

    album_df = pd.read_csv(f'data-sm_vs_yg/albums/{artist_id_}_album_2_pages.csv')
    album_id_lst = album_df.album_id.to_list()

    album_id = []
    song_id = []
    song_title = []

    for album_id_ in album_id_lst:
        url_album = f'https://music.bugs.co.kr/album/{album_id_}'
        request_album = requests.get(url_album)
        html_album = request_album.text
        soup_album = BeautifulSoup(html_album, 'html.parser')

        album_song_list=soup_album.find('table', attrs = {'class': 'list trackList byAlbum'})

        for elem in album_song_list.find_all('p', attrs={'class':'title'}):
            try:            
                song_id.append(elem.select('a')[0]['onclick'].split('.listen(')[1].split(',')[0][1:-1])
                song_title.append(elem.get_text().strip('\n'))
                album_id.append(album_id_)

            except:
                album_id.append(album_id_)
                song_id.append('-')
                song_title.append(elem.get_text().replace('\n', ' ').strip())

    column_list = {
            'album_id': album_id,
            'song_id': song_id,      
            'song_title': song_title
            }

    df1 = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df1 = df1.transpose() # 행과 열 전환 

    return df1.to_csv(f'data-sm_vs_yg/albums/{artist_id_}_album_and_songs_2_page.csv') # 편집자 실행


In [373]:
album_and_songs2(64843)

In [374]:
def merge_album_songs_df2(artist_id_):
    al_ = pd.read_csv(f'data-sm_vs_yg/albums/{artist_id_}_album_2_pages.csv')
    al_and_song = pd.read_csv(f'data-sm_vs_yg/albums/{artist_id_}_album_and_songs_2_page.csv')
    merged_df = al_and_song.merge(al_, how='left', on='album_id')[['artist_id', 'artist_name', 'album_id', 'album_title', 'song_id', 'song_title', 'release_date']]
    return merged_df.to_csv(f'data-sm_vs_yg/tracks/{artist_id_}_merged_2.csv')

In [375]:
merge_album_songs_df2(64843)

In [378]:
def lyrics_scraping2(artist_id_):
    
    song_df = pd.read_csv(f'data-sm_vs_yg/tracks/{artist_id_}_merged_2.csv')
    song_df = song_df[song_df['song_id']!='-']
    song_id_lst1 = list(set(song_df.song_id.to_list()))

    song_id_lst = []
    lyric_lst = []
    artist_name_lst = []


    for i in song_id_lst1:
        l_url = f'https://music.bugs.co.kr/track/{i}'
        l_request = requests.get(l_url)
        l_html = l_request.text
        l_soup = BeautifulSoup(l_html, 'html.parser')
        
        try:
            artist_name = l_soup.select('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(1) > td > a')[0].getText().strip('\r\n\t')
        except:
            artist_name = '-'

        lyric = str(l_soup.select(
                '#container > section.sectionPadding.contents.lyrics > div.innerContainer > div.lyricsContainer > p > xmp'))
            # 가사 데이터가 있는 html 구조 #container > section.sectionPadding.contents.lyrics > div > div > xmp
        lyric = re.sub('<.+?>', '', lyric, 0)

        song_id_lst.append(i)
        artist_name_lst.append(artist_name)
        lyric_lst.append(lyric)


    column_list = { # dict-key
            'song_id': song_id_lst,
            'song_artist': artist_name_lst,
            'song_lyrics': lyric_lst,
        }

    df = pd.DataFrame.from_dict(column_list, orient='index') # dict를 dataframe 형식으로 변환
    df = df.transpose() # 행과 열 전환


    return df.to_csv(f'data-sm_vs_yg/lyric/{artist_id_}_lyrics_2.csv') # 편집자 실행


In [379]:
lyrics_scraping2(64843)

In [381]:
# tvxq_merged_1 = pd.read_csv('data-sm_vs_yg/tracks/64843_merged.csv')
# tvxq_merged_2 = pd.read_csv('data-sm_vs_yg/tracks/64843_merged_2.csv')
# concat_tvxq = pd.concat([tvxq_merged_1, tvxq_merged_2])
# concat_tvxq.to_csv('data-sm_vs_yg/tracks/64843_merged.csv')

In [393]:
# tvxq_lyric_1 = pd.read_csv('data-sm_vs_yg/lyric/64843_lyrics.csv')
# tvxq_lyric_2 = pd.read_csv('data-sm_vs_yg/lyric/64843_lyrics_2.csv')
# concat_tvxq_l = pd.concat([tvxq_lyric_1, tvxq_lyric_2])
# concat_tvxq_l.to_csv('data-sm_vs_yg/lyric/64843_lyrics.csv')

In [398]:
concat_tvxq_l

,Unnamed: 0,song_id,song_artist,song_lyrics
0,0,31240904,동방신기 (TVXQ!),[足を止めて 見上げたのは 澄み切った碧空\r\n希望を瞳に 焼き付けたら また歩いていこう...
1,1,31321810,동방신기 (TVXQ!),[あの空も\tTシャツも 雨の音も 大好きだった\n潮風も 街路樹も 君の夏 大好きだった ...
2,2,1935678,동방신기 (TVXQ!),[かえりたい長い夜に 僕の胸はまだ迷っている\r\n카에리따이 나가이 요루니 보쿠노 무네...
3,3,3222872,동방신기 (TVXQ!),[]
4,4,1780963,동방신기 (TVXQ!),[]
...,...,...,...,...
337,337,507874,동방신기 (TVXQ!),[하루만 니 방의 침대가 되고 싶어 Oh Baby\r\n더 따스히 포근히 내 품에 ...
338,338,507875,동방신기 (TVXQ!),[사랑이겠죠 이런내마음은\r\n숨기려해도 그새 입가에 미소만\r\n\r\n하루도 안...
339,339,507876,동방신기 (TVXQ!),[Oh Holy Night(Feat. 보아)-동방신기\r\n\r\n\r\nO hol...
340,340,507877,동방신기 (TVXQ!),[사랑이겠죠 이런내마음은\r\n숨기려해도 그새 입가에 미소만\r\n\r\n하루도 안...


## Merge - 2 ( Songs & Lyrics) -> Final

In [48]:
def merge_df(artist_id_):
    lyrics_df = pd.read_csv(f'data/lyrics/{artist_id_}_lyrics.csv')
    albums_df = pd.read_csv(f'data/tracks/{artist_id_}_merged.csv')
    lyrics_df['song_id'] = lyrics_df['song_id'].astype('str')
    albums_df['song_id'] = albums_df['song_id'].astype('str')
    merged_df = albums_df.merge(lyrics_df, how='left', on='song_id')
    merged_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)
    return merged_df.to_csv(f'data/final/{artist_id_}_final.csv')

In [49]:
%%time
for ids in group_ids:
    merge_df(ids)
for ids in solo_ids:
    merge_df(ids)

CPU times: user 610 ms, sys: 106 ms, total: 716 ms
Wall time: 753 ms


## Concat (All artists)


In [50]:
groups_all_df_lst = []
solo_all_df_lst = []

for ids in group_ids:
    groups_all_df_lst.append(pd.read_csv(f'data/final/{ids}_final.csv'))
for ids in solo_ids:
    solo_all_df_lst.append(pd.read_csv(f'data/final/{ids}_final.csv'))    

In [52]:
groups_df = pd.concat(groups_all_df_lst)
solo_df = pd.concat(solo_all_df_lst)

In [57]:
solo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2285 entries, 0 to 82
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2285 non-null   int64 
 1   artist_id     2285 non-null   int64 
 2   artist_name   2285 non-null   object
 3   album_id      2285 non-null   int64 
 4   album_title   2285 non-null   object
 5   song_id       2285 non-null   object
 6   song_title    2285 non-null   object
 7   release_date  2285 non-null   object
 8   song_artist   2194 non-null   object
 9   song_lyrics   2194 non-null   object
dtypes: int64(3), object(7)
memory usage: 196.4+ KB


In [95]:
groups_df['release_date'] = pd.to_datetime(groups_df['release_date'])
groups_df['year'] = pd.DatetimeIndex(groups_df['release_date']).year

In [96]:
def date_len(text):
    if text == '2000.12':
        return '2000.12.01'
    if text == '2001.12':
        return '2001.12.01'
    else:
        return text

In [97]:
solo_df['release_date'] = solo_df['release_date'].apply(date_len)
solo_df.head(2)

,Unnamed: 0,artist_id,artist_name,album_id,album_title,song_id,song_title,release_date,song_artist,song_lyrics,date2,year
0,0,80049126,아이유(IU),4070675,강승원 이집 PART.3 - Mother Nature (H₂O),6145236,Mother Nature (H₂O),2022.01.27,아이유(IU),[꿈. 기도. 달빛 되어 내린 눈물\r\n너. 나. 이제 잠을 청해 본다\r\n잠자...,2022-01-27,2022
1,1,80049126,아이유(IU),4069567,조각집,6141496,드라마,2021.12.29,아이유(IU),[나도 한때는 그이의 손을 잡고\r\n내가 온 세상 주인공이 된 듯\r\n꽃송이의 ...,2021-12-29,2021


In [98]:
solo_df['release_date'] = pd.to_datetime(solo_df['release_date'])
solo_df['year'] = pd.DatetimeIndex(solo_df['release_date']).year

In [100]:
all_df = pd.concat([groups_df, solo_df]).drop(columns=['Unnamed: 0', 'date2'])
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6034 entries, 0 to 82
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   artist_id     6034 non-null   int64         
 1   artist_name   6034 non-null   object        
 2   album_id      6034 non-null   int64         
 3   album_title   6034 non-null   object        
 4   song_id       6034 non-null   object        
 5   song_title    6034 non-null   object        
 6   release_date  6034 non-null   datetime64[ns]
 7   song_artist   5695 non-null   object        
 8   song_lyrics   5695 non-null   object        
 9   year          6034 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(6)
memory usage: 495.0+ KB


In [101]:
all_df

,artist_id,artist_name,album_id,album_title,song_id,song_title,release_date,song_artist,song_lyrics,year
0,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704240,FOREVER 1 (Matisse & Sadko Remix),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
1,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704241,FOREVER 1 (Aiobahn Remix),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
2,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704242,FOREVER 1 (Mar Vista Remix),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
3,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704243,FOREVER 1 (Matisse & Sadko Remix) (Extended Ver.),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
4,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704244,FOREVER 1 (Aiobahn Remix) (Extended Ver.),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
...,...,...,...,...,...,...,...,...,...,...
78,80046074,Various Artists,672309,마이 리틀 포니 : 더 무비 (My Little Pony : The Movie) OST,-,[권리없는 곡] Neighsayer,2017-09-22,NaN,NaN,2017
79,80046074,CL,20011777,Hello Bitches,30085329,Hello Bitches,2015-12-05,CL,[엉덩이 빵빵빵 \r\n남자들은 탐탐탐 \r\n내 입술은 냠냠냠 \r\n어딜가도 당...,2015
80,80046074,Diplo(디플로),507020,Doctor Pepper,-,[19금] Doctor Pepper,2015-05-29,NaN,NaN,2015
81,80046074,CL,370709,나쁜 기집애,2987681,나쁜 기집애,2013-05-28,CL,[난 나쁜 기집애 나난 나쁜 기집애 난 나쁜 기집애 \r\nWhere All My ...,2013


In [102]:
all_df.to_csv('major_girlgroups_final.csv', index=False)

In [103]:
pd.read_csv('major_girlgroups_final.csv')

,artist_id,artist_name,album_id,album_title,song_id,song_title,release_date,song_artist,song_lyrics,year
0,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704240,FOREVER 1 (Matisse & Sadko Remix),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
1,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704241,FOREVER 1 (Aiobahn Remix),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
2,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704242,FOREVER 1 (Mar Vista Remix),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
3,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704243,FOREVER 1 (Matisse & Sadko Remix) (Extended Ver.),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
4,80026432,소녀시대 (GIRLS' GENERATION),20528353,iScreaM Vol.19 : FOREVER 1 Remixes,32704244,FOREVER 1 (Aiobahn Remix) (Extended Ver.),2022-11-17,소녀시대 (GIRLS' GENERATION),[FOREVER 1\r\nIt's love It's love\r\nWe're not...,2022
...,...,...,...,...,...,...,...,...,...,...
6029,80046074,Various Artists,672309,마이 리틀 포니 : 더 무비 (My Little Pony : The Movie) OST,-,[권리없는 곡] Neighsayer,2017-09-22,NaN,NaN,2017
6030,80046074,CL,20011777,Hello Bitches,30085329,Hello Bitches,2015-12-05,CL,[엉덩이 빵빵빵 \r\n남자들은 탐탐탐 \r\n내 입술은 냠냠냠 \r\n어딜가도 당...,2015
6031,80046074,Diplo(디플로),507020,Doctor Pepper,-,[19금] Doctor Pepper,2015-05-29,NaN,NaN,2015
6032,80046074,CL,370709,나쁜 기집애,2987681,나쁜 기집애,2013-05-28,CL,[난 나쁜 기집애 나난 나쁜 기집애 난 나쁜 기집애 \r\nWhere All My ...,2013


In [ ]:
# { #from bugs
# 'artist_name':,
# 'artist_id':,
# 'debut_year':,
# 'album_name':,
#  'album_release_date':,
# 'album_id':,
# 'track_name':,
# 'track_id':,
#  'track_length':,
# 'Lyrics':,

# #from youtube
# 'youtube_viewcount':,
#  'school_girl':,
    
# #from kaggle dataset
# 'dancability':,
# ''}